# Build `data/prices.csv` from Yahoo Finance (Colab workflow)

Use this notebook when the local environment cannot reach Yahoo Finance. It
downloads all tickers referenced in `data/weights/weight_week*/` and converts
Korean listings (tickers ending in `.KS`) from KRW to USD using the daily
`KRW=X` exchange rate. Steps:

1. Mount Google Drive so the notebook can access your existing `data/weights/` directory.
2. Update the `WEIGHTS_DIR_PATH` variable to point to the folder inside Drive.
3. Run the remaining cells in order.
4. Download the generated `data/prices.csv` and copy it back into the repo.

In [ ]:
!pip install --quiet pandas yfinance

In [1]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
WEIGHTS_DIR_PATH = "/content/drive/MyDrive/SKKU/ChatGPT_vs_human/data/weights"  # <-- Change this!

import json
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, Mapping

import pandas as pd
import yfinance as yf

CASH_KEYWORDS = {"cash", "money"}
IGNORE_META = {
    "week_of", "week", "date", "notes", "note", "comment", "comments",
    "team", "name", "id", "portfolio_name",
}
CONTAINER_KEYS = {"portfolio", "weights", "allocations", "allocation", "holdings"}

def load_weight_mapping(path: Path) -> Dict[str, float]:
    text = path.read_text().strip()
    if not text:
        raise ValueError(f"Empty weight file: {path}")
    candidates = [text]
    if "=" in text:
        candidates.append(text.split("=", 1)[1].strip())
    brace_start = text.find("{")
    brace_end = text.rfind("}")
    if brace_start != -1 and brace_end != -1 and brace_end > brace_start:
        candidates.append(text[brace_start:brace_end + 1])
    for candidate in candidates:
        cleaned = candidate.strip().rstrip(";.")
        try:
            data = json.loads(cleaned)
        except json.JSONDecodeError:
            continue
        weights = extract_weight_mapping(data)
        if weights:
            return weights
    raise ValueError(f"Unable to parse weight file as JSON: {path}")

def extract_weight_mapping(obj) -> Dict[str, float]:
    def normalise_ticker(ticker: str) -> str:
        t_clean = str(ticker).strip()
        if t_clean.lower() in CASH_KEYWORDS:
            return "CASH"
        return t_clean

    def from_mapping(data: Mapping) -> Dict[str, float]:
        acc: Dict[str, float] = {}
        for key, value in data.items():
            key_str = str(key).strip()
            key_lower = key_str.lower()
            if key_lower in IGNORE_META:
                continue
            if key_lower in CONTAINER_KEYS and isinstance(value, (Mapping, list)):
                acc.update(extract_weight_mapping(value))
                continue
            if isinstance(value, Mapping):
                weight_val = find_weight_field(value)
                if weight_val is not None:
                    acc[normalise_ticker(key_str)] = weight_val
                else:
                    acc.update(extract_weight_mapping(value))
                continue
            if isinstance(value, (int, float)):
                acc[normalise_ticker(key_str)] = float(value)
                continue
            if isinstance(value, str):
                try:
                    acc[normalise_ticker(key_str)] = float(value)
                except ValueError:
                    pass
        return acc

    if isinstance(obj, Mapping):
        return from_mapping(obj)
    if isinstance(obj, list):
        merged: Dict[str, float] = {}
        for item in obj:
            merged.update(extract_weight_mapping(item))
        return merged
    return {}

def find_weight_field(candidate: Mapping) -> float | None:
    for key, value in candidate.items():
        if str(key).strip().lower() == "weight":
            try:
                return float(value)
            except (TypeError, ValueError):
                return None
    return None

def gather_tickers(weights_root: Path) -> list[str]:
    tickers = set()
    for path in weights_root.glob("weight_week*/*.json"):
        weights = load_weight_mapping(path)
        tickers.update(t for t in weights if t != "CASH")
    if not tickers:
        raise ValueError("No tickers found – ensure data/weights is uploaded.")
    return sorted(tickers)

In [3]:
weights_dir = Path(WEIGHTS_DIR_PATH)
if not weights_dir.exists():
    raise FileNotFoundError(f"weights_dir does not exist: {weights_dir}")
tickers = gather_tickers(weights_dir)
print(f"Found {len(tickers)} tickers across weekly submissions.")
tickers[:10]

Found 280 tickers across weekly submissions.


['000660.KS',
 '003490.KS',
 '006800.KS',
 '032830.KS',
 '035420.KS',
 '042660.KS',
 '114800.KS',
 '185750.KS',
 '278470.KS',
 '411060.KS']

In [4]:
# Adjust dates as needed
start_date = "2025-09-30"
end_date = "2025-10-30"

chunk_size = 50
frames = []
yf_end = (datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

for i in range(0, len(tickers), chunk_size):
    batch = tickers[i:i + chunk_size]
    print(f"Downloading {len(batch)} tickers ({batch[0]} … {batch[-1]})")
    raw = yf.download(
        tickers=batch,
        start=start_date,
        end=yf_end,
        interval="1d",
        group_by="ticker",
        auto_adjust=False,
        actions=False,
        progress=False,
        threads=True,
    )
    if raw.empty:
        continue
    if isinstance(raw.columns, pd.MultiIndex):
        if "Close" in raw.columns.get_level_values(0):
            close = raw["Close"].copy()
        else:
            close_frames = []
            for ticker in batch:
                if ticker in raw.columns.get_level_values(0):
                    sub = raw[ticker]
                    if "Close" in sub.columns:
                        close_frames.append(sub["Close"].rename(ticker))
            close = pd.concat(close_frames, axis=1) if close_frames else pd.DataFrame()
    else:
        close = raw.copy()
    if not close.empty:
        frames.append(close)

if not frames:
    raise RuntimeError("No price data returned. Verify tickers and network access.")

prices = pd.concat(frames, axis=1)
prices = prices.loc[~prices.index.duplicated()].sort_index()
prices.index = prices.index.tz_localize(None)
prices.index.name = "Date"

non_crypto_cols = [col for col in prices.columns if not col.endswith("-USD")]
if non_crypto_cols:
    only_crypto_mask = prices[non_crypto_cols].isna().all(axis=1)
    if only_crypto_mask.any():
        drop_count = int(only_crypto_mask.sum())
        print(f"Dropping {drop_count} rows with only crypto quotes (e.g., weekend sessions).")
        prices = prices.loc[~only_crypto_mask]

prices = prices.ffill().dropna(how="all")
print(prices.shape)
prices.head()

Dropping 8 rows with only crypto quotes (e.g., weekend sessions).
(22, 280)


,000660.KS,003490.KS,006800.KS,032830.KS,035420.KS,042660.KS,114800.KS,185750.KS,278470.KS,411060.KS,...,WMB,WMT,WRB,WYNN,XEL,XLF,XLK,XLV,ZEPP,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2025-09-30,347500.0,22750.0,21350.0,156600.0,268500.0,110300.0,3160.0,82100.0,250000.0,27400.0,...,63.349998,103.059998,76.620003,128.270004,80.650002,53.869999,281.859985,139.169998,45.779999,146.320007
2025-10-01,360000.0,22700.0,21000.0,155100.0,254500.0,110400.0,3130.0,81400.0,261000.0,26990.0,...,63.689999,101.959999,75.410004,132.029999,80.309998,53.389999,284.619995,143.470001,49.930000,146.949997
2025-10-02,395500.0,22900.0,21700.0,160600.0,253000.0,110650.0,3030.0,82300.0,255000.0,26360.0,...,64.059998,101.699997,76.180000,133.339996,79.599998,53.349998,286.160004,143.190002,49.630001,146.490005
2025-10-03,395500.0,22900.0,21700.0,160600.0,253000.0,110650.0,3030.0,82300.0,255000.0,26360.0,...,64.480003,102.070000,76.860001,123.660004,80.260002,53.720001,284.720001,144.820007,49.700001,146.419998
2025-10-06,395500.0,22900.0,21700.0,160600.0,253000.0,110650.0,3030.0,82300.0,255000.0,26360.0,...,63.580002,102.699997,76.980003,124.589996,81.000000,53.700001,287.600006,144.119995,56.119999,145.360001


In [9]:
kr_tickers = [t for t in prices.columns if t.endswith(".KS")]
if kr_tickers:
    print(f"Converting {len(kr_tickers)} Korean tickers from KRW to USD using KRW=X.")
    fx = yf.download(
        tickers="KRW=X",
        start=start_date,
        end=yf_end,
        interval="1d",
        auto_adjust=False,
        progress=False,
    )
    if fx.empty:
        raise RuntimeError("FX rate download returned no data; check date range orconnectivity.")

    if isinstance(fx.columns, pd.MultiIndex):
        fx_close = fx["Close"]
        if isinstance(fx_close, pd.Series):
            fx_series = fx_close
        else:
            if "KRW=X" in fx_close.columns:
                fx_series = fx_close["KRW=X"]
            else:
                fx_series = fx_close.squeeze()
    else:
        fx_close = fx["Close"]
        fx_series = fx_close.squeeze()

    if not isinstance(fx_series, pd.Series):
        fx_series = pd.Series(fx_series)

    fx_series = fx_series.rename("KRW=X").sort_index().tz_localize(None)
    fx_series = fx_series.reindex(prices.index).ffill()
    if fx_series.isna().any():
        raise ValueError("Missing FX rates after forward fill – adjust date range or inspect data.")

    for ticker in kr_tickers:
        prices[ticker] = prices[ticker] / fx_series
else:
    print("No Korean tickers detected – skipping FX conversion.")

prices.head()


Converting 10 Korean tickers from KRW to USD using KRW=X.


,000660.KS,003490.KS,006800.KS,032830.KS,035420.KS,042660.KS,114800.KS,185750.KS,278470.KS,411060.KS,...,WMB,WMT,WRB,WYNN,XEL,XLF,XLK,XLV,ZEPP,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2025-09-30,248.333139,16.257781,15.257302,111.910704,191.877547,78.823439,2.258224,58.670937,178.656934,19.580800,...,63.349998,103.059998,76.620003,128.270004,80.650002,53.869999,281.859985,139.169998,45.779999,146.320007
2025-10-01,256.565580,16.177885,14.966326,110.537004,181.377611,78.680111,2.230695,58.012328,186.010046,19.235292,...,63.689999,101.959999,75.410004,132.029999,80.309998,53.389999,284.619995,143.470001,49.930000,146.949997
2025-10-02,282.133238,16.335907,15.479877,114.565355,180.479669,78.933104,2.161476,58.709394,181.906386,18.804127,...,64.059998,101.699997,76.180000,133.339996,79.599998,53.349998,286.160004,143.190002,49.630001,146.490005
2025-10-03,281.368496,16.291627,15.437917,114.254818,179.990466,78.719151,2.155617,58.550258,181.413316,18.753157,...,64.480003,102.070000,76.860001,123.660004,80.260002,53.720001,284.720001,144.820007,49.700001,146.419998
2025-10-06,281.282437,16.286644,15.433196,114.219872,179.935415,78.695074,2.154958,58.532350,181.357829,18.747421,...,63.580002,102.699997,76.980003,124.589996,81.000000,53.700001,287.600006,144.119995,56.119999,145.360001


In [10]:
output_path = Path("data/prices.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)
prices.to_csv(output_path, float_format="%.6f")
output_path

PosixPath('data/prices.csv')

In [11]:
from google.colab import files

files.download("data/prices.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>